**Trabalho SIA 2º Bimestre**

Ana Carolina Jungbluth e Isabella Rodrigues Gon

**Importando o DataSet**

In [ ]:
import pandas as pd
dados = pd.read_csv('/content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [ ]:
dados

**Analisando o DataSet**

In [ ]:
dados.info()

In [ ]:
dados.describe().T

In [ ]:
dados.head(5)

**Removendo valores nulos**

In [ ]:
dados = dados.dropna(how='any')

In [ ]:
dados.head(10)

**Transformando a Classe para Binário**

In [ ]:
types_of_label = dados[' Label'].unique()

In [ ]:
types_of_label

array(['BENIGN', 'DDoS'], dtype=object)

In [ ]:
replace_str = {}
for i,value in enumerate(types_of_label, start=0):
  replace_str[value]=i
dados[' Label'] = dados[' Label'].map(replace_str)
dados

In [ ]:
dados.head(5)

In [ ]:
dados[' Label'].unique()

array([0, 1])

**Balanceamento**

In [ ]:
def count_class(df,types_label):
  types_label = df[' Label'].unique()
  for atual_class in types_label:
    _class = df[df[' Label']==atual_class]
    print("Tamanho da classe {}:  {}".format(atual_class,len(_class)))
count_class(dados,types_of_label)

Tamanho da classe 0:  97714
Tamanho da classe 1:  128027


In [ ]:
dados.columns

**Transformando o IP em numérico**



In [ ]:
import ipaddress
# Aplicar a conversão para formato numérico para a coluna 'Source IP' e 'Destination IP'
dados['Source IP Numerico'] = dados[' Source IP'].apply(lambda x: int(ipaddress.IPv4Address(x)) if isinstance(x, str) else x)
dados['Destination IP Numerico'] = dados[' Destination IP'].apply(lambda x: int(ipaddress.IPv4Address(x)) if isinstance(x, str) else x)


In [ ]:
dados.head()

In [ ]:
dados.columns

In [ ]:
dados = dados.drop(' Destination IP', axis=1)

In [ ]:
dados = dados.drop('Flow ID', axis=1)

In [ ]:
dados = dados.drop(' Source IP', axis=1)

In [ ]:
dados.head()

**Seleção de atributos por correlação**

In [ ]:
correlation = dados.corr()
low_correlation_coluns = correlation.loc[(correlation[' Label'] < .3) & (correlation[' Label'] > -.3)].index
len(low_correlation_coluns)

<ipython-input-50-0db1f9a04da6>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = dados.corr()


46

In [ ]:
dados_sem_low_correlation = dados.drop(columns=low_correlation_coluns)

In [ ]:
dados_sem_low_correlation.head()

In [ ]:
dados2 = dados_sem_low_correlation

In [ ]:
dados2.head()

**Transformando o atributo 'Timestamp' em 'Date' e 'Time'**

Como o dataset só está situado em um dia 07/07/2017, então o novo atributo 'Date' não se torna relevante, sendo assim o único atributo que mantivemos em relação a data e hora foi o 'Time', excluindo tanto o 'Timestamp', quanto o 'Date'.

In [ ]:
dados2['Date'] = pd.to_datetime(dados[' Timestamp']).dt.date
dados2['Time'] = pd.to_datetime(dados[' Timestamp']).dt.time


In [ ]:
dados2.head()

In [ ]:
dados2 = dados2.drop(' Timestamp', axis=1)

In [ ]:
dados2 = dados2.drop('Date', axis=1)

In [ ]:
dados2.head()

In [ ]:
def time_to_seconds(time_obj):
    total_seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
    return total_seconds

dados2['Time_Numeric'] = dados2['Time'].apply(time_to_seconds)


In [ ]:
dados2.head()

In [ ]:
dados2 = dados2.drop('Time', axis=1)

In [ ]:
dados2.head()

**Remoção de outliers utilizando IsolationForest**

In [ ]:
from sklearn.ensemble import IsolationForest

model = IsolationForest()
model.fit(dados2)
outliers = model.predict(dados2)
dados_sem_outliers = dados2[outliers == 1]


In [ ]:
dados2.head()

In [ ]:
dados_sem_outliers

In [ ]:
dados3 = dados_sem_outliers

In [ ]:
dados3

**Variáveis de treinamento e teste**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

x = dados3.drop([' Label'], axis=1)
y = dados3[' Label']

x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, random_state=10, test_size=0.3)

In [ ]:
x

In [ ]:
y

In [ ]:
x_treinamento.shape, x_teste.shape, y_treinamento.shape, y_teste.shape

((127444, 37), (54620, 37), (127444,), (54620,))

**Tabela de informações**

In [ ]:
def train_model(model, X_treinamento, y_treinamento):
    treinamento_model = model.fit(X_treinamento, y_treinamento)
    return treinamento_model

In [ ]:
def predict_with_model(treinamento_model, x_teste):
    y_pred = treinamento_model.predict(x_teste)
    return y_pred

In [ ]:
def execute_test(model, X_treinamento = x_treinamento, y_treinamento= y_treinamento, X_teste= x_teste):
    treinamento_model = train_model(model, X_treinamento, y_treinamento)
    y_pred = predict_with_model(treinamento_model, X_teste)
    return y_pred

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def show_test(y_pred, y_teste=y_teste):
    accuracy = accuracy_score(y_teste, y_pred)
    print("Acurácia:", accuracy*100,"%")

    report = classification_report(y_teste, y_pred, output_dict=True)
    print("\n\t     Relatório de Classificação")
    print("-----------------------------------------------------")
    print(f"   {'Classe':<10}{'Precisão':<10}{'Recall':<10}{'F1-Score':<10}{'Suporte':<10}")
    print("-----------------------------------------------------")
    for cls, metrics in report.items():
        if cls.isdigit():
            print(f"    {cls:<10}{metrics['precision']:<10.2f}{metrics['recall']:<10.2f}{metrics['f1-score']:<10.2f}{metrics['support']:<10}")
    print("-----------------------------------------------------")

    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_teste, y_pred), annot=True, fmt='d', cmap='Purples')
    plt.xlabel('Predict')
    plt.ylabel('True')
    plt.title('Matriz de Confusão')
    plt.show()


**Árvore CART**

In [ ]:
from sklearn import tree

tree = tree.DecisionTreeClassifier()
y_pred = execute_test(tree, x_treinamento, y_treinamento, x_teste)
show_test(y_pred)

**SVM**

Pré-processamento específico SVM

In [ ]:
x_testaux = x_teste
x_treinaux = x_treinamento
y_testaux = y_teste
y_treinaux = y_treinamento

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

svm_model = SVC()

scaler = StandardScaler()
x_treinamento = scaler.fit_transform(x_treinamento)

svm_model.fit(x_treinamento, y_treinamento)

x_teste = scaler.transform(x_teste)
y_pred_svm = svm_model.predict(x_teste)

In [ ]:
x_treinamento

In [ ]:
x_teste

In [ ]:
y_pred_svm = execute_test(svm_model, X_treinamento=x_treinamento, y_treinamento=y_treinamento, X_teste=x_teste)
show_test(y_pred_svm)

**Alterando o valor do kernel**

In [ ]:
x_treinamento, x_teste

In [ ]:
svm_model = SVC(kernel='poly')

y_pred_svm = execute_test(svm_model, X_treinamento=x_treinamento, y_treinamento=y_treinamento, X_teste=x_teste)
show_test(y_pred_svm)

**KNN**

In [ ]:
x_treinamento, x_teste

**Padrão scikit-learn 5**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
y_pred_knn = execute_test(knn_model, X_treinamento=x_treinamento, y_treinamento=y_treinamento, X_teste=x_teste)
show_test(y_pred_knn)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=1)
y_pred_knn = execute_test(knn_model, X_treinamento=x_treinamento, X_teste=x_teste)
show_test(y_pred_knn)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=14)
y_pred_knn = execute_test(knn_model, X_treinamento=x_treinamento, X_teste=x_teste)
show_test(y_pred_knn)

In [ ]:
knn_model = KNeighborsClassifier(metric = 'manhattan')
y_pred_knn = execute_test(knn_model, X_treinamento=x_treinamento, X_teste=x_teste)
show_test(y_pred_knn)

In [ ]:
knn_model = KNeighborsClassifier(metric = 'chebyshev')
y_pred_knn = execute_test(knn_model, X_treinamento=x_treinamento, X_teste=x_teste)
show_test(y_pred_knn)

**Gaussian Naive Bayes**

In [ ]:
x_treinamento, x_teste

In [ ]:
from sklearn.naive_bayes import GaussianNB

naive_bayes_model = GaussianNB()
y_pred_gnb = execute_test(naive_bayes_model, X_treinamento=x_treinamento, X_teste=x_teste)
show_test(y_pred_gnb)


**Categorical Naive Bayes**

In [ ]:
dados3.info()

In [ ]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
x_treinamento_discretized = discretizer.fit_transform(x_treinaux)

x_teste_discretized = discretizer.transform(x_testaux)

categorical_nb_model = CategoricalNB()

y_pred_categorical_nb = execute_test(categorical_nb_model, X_treinamento=x_treinamento_discretized, X_teste=x_teste_discretized)

show_test(y_pred_categorical_nb, y_teste)


In [ ]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
x_treinamento_discretized = discretizer.fit_transform(x_treinaux)

x_teste_discretized = discretizer.transform(x_testaux)

categorical_nb_model = CategoricalNB()

y_pred_categorical_nb = execute_test(categorical_nb_model, X_treinamento=x_treinamento_discretized, X_teste=x_teste_discretized)

show_test(y_pred_categorical_nb, y_teste)

**Perceptron**

In [ ]:
from sklearn.linear_model import Perceptron

perceptron_model = Perceptron()
y_pred_percep = execute_test(perceptron_model, X_treinamento=x_treinamento, X_teste=x_teste)
show_test(y_pred_percep)


**Regressão Linear**

In [ ]:
dadosReg = pd.read_csv('/content/BostonHousing.csv')

In [ ]:
dadosReg

In [ ]:
from sklearn.ensemble import IsolationForest

model = IsolationForest()
model.fit(dadosReg)
outliers = model.predict(dadosReg)
dados_sem_outliers = dadosReg[outliers == 1]

In [ ]:
dadosReg2 = dados_sem_outliers

In [ ]:
dadosReg2

In [ ]:
from sklearn.model_selection import train_test_split

xReg = dadosReg2.drop(['medv'], axis=1)
yReg = dadosReg2['medv']

x_treinamenReg, x_testeReg, y_treinamenReg, y_testeReg = train_test_split(xReg, yReg, random_state=10, test_size=0.3)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

modelo_regressao = LinearRegression()

modelo_regressao.fit(x_treinamenReg, y_treinamenReg)

y_pred = modelo_regressao.predict(x_testeReg)

erro_quadratico = mean_squared_error(y_testeReg, y_pred)
print(f"Erro Quadrático Médio: {erro_quadratico}")